# Cases

Esta tabla contiene la información de las sentencias de tipo Match. --> ABEL COMPLETAR <--

## IMPORTANTE
Todos las instancias de la tabla Cases tienen expertise_level = EXPERT, con lo que solo se hara este analisis.  

## 1 - Obtencion de datos

In [ ]:
import matplotlib.pyplot as plt
from notebooks_utils import *

full_table = get_data('cases')

# Table name, features and target.
TABLE_NAME = 'cases'
TABLE_FEATURES = ['case__number_of_cases', 'case__guards', 'case__average_match_value', 'case__average_body_count', 'case__average_match_singleton', 'case__average_match_sequence', 'case__average_match_mapping', 'case__average_match_class', 'case__average_match_star', 'case__average_match_as', 'case__average_match_or', 'case__expertise_level']
TABLE_TARGET = 'case__expertise_level'

# Load features and target.
X, y = full_table[TABLE_FEATURES], full_table[[TABLE_TARGET]].iloc[:,0]

# Print information about the loaded table.
print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')

print(f'As we can see the downloaded data contains a total of {X.shape[0]} instances. For each instance we have {X.shape[1]} attributes.')

## 2 - Exploracion de datos

Una vez tenemos nuestra tabla en un dataframe el siguiente paso es explorarla para ver qué tipo de información contiene.

In [ ]:
print(X.info())
print('=============')
print(y.info())

Cómo podemos ver la tabla está compuesta por 11 variables numéricas y una de tipo objeto. 

### 2.1 - Balance de clases

In [ ]:
instances_for_class_low = len(full_table[full_table[TABLE_TARGET] == "BEGINNER"])
instances_for_class_high = len(full_table[full_table[TABLE_TARGET] == "EXPERT"])

print(f"The dataset contains {instances_for_class_low/len(full_table)*100:.4}% instances for BEGINNER class and {instances_for_class_high/len(full_table)*100:.4}% for EXPERT class.")

### 2.2 - Duplicados
Miramos si la tabla tiene entradas duplicadas.

In [ ]:
number_of_duplicated_entries = sum(full_table.duplicated(subset=TABLE_FEATURES + [TABLE_TARGET]))
duplicated_entries_pct = number_of_duplicated_entries / len(full_table) * 100
print(f"The dataset contains [{duplicated_entries_pct:.4}%] of duplicated entries.")

### 2.3 - Valores Nulos 
Miramos si alguna de las variables que contiene la tabla contiene algún valor que sea nulo.

In [ ]:
X.isnull().sum()

Aunque una columna no contenga valores nulos podría ser que contenga valores vacíos. Si los hubiese la siguiente función los mostraría.

In [ ]:
print_empty_cols(X)

### 2.4 - Describimos los valores de las variables de la tabla.

In [ ]:
np.transpose(X.describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int']))

Vamos a discretizar las variables numericas, agrupando conjuntos de valores en categorias, para hacer un análisis de los datos. Para cada variable es necesaario ver la distribucion de lo valores para hacer los bins (categorias).

In [ ]:
# DISCRETIZATION
X_copy = X.copy()

discretized_columns = {
    "case__number_of_cases": [(0.0, 2.0), (2.0, 3.0), (3.0, 4.0), (4.0, inf)],  # min: 2.0 max: 51.0 
    "case__guards": [(0.0, 0.0)],  # min: 0.0 max: 0.0 
    "case__average_match_value": [(0.0, 0.5), (0.5, 0.67), (0.67, inf)],  # min: 0.5 max: 1.0 
    "case__average_body_count": [(0.0, 1.15), (1.15, 2.33), (2.33, 2.5), (2.5, inf)],  # min: 1 max: 3.0 
    "case__average_match_singleton": [(0.0, 0.0)],  # min: 0 max: 0 
    "case__average_match_sequence": [(0.0, 0.0)],  # min: 0 max: 0 
    "case__average_match_mapping": [(0.0, 0.0)],  # min: 0 max: 0 
    "case__average_match_class": [(0.0, inf)],  # min: 0 max: 0.0526 
    "case__average_match_star": [(0.0, 0.0)],  # min: 0 max: 0 
    "case__average_match_or": [(0.0, 0.0)],  # min: 0 max: 0 
    "case__average_match_as": [(0.0, 0.0), (0.0, 0.34), (0.34, inf)],  # min: 0 max: 0.5
}

discretize_columns(X_copy, discretized_columns)
    
# SINGLE FEATURE
print("--- SINGLE FEATURE ---")
print(get_statistics(X_copy, ['case__number_of_cases'], 10))
print(get_statistics(X_copy, ['case__guards'], 10))
print(get_statistics(X_copy, ['case__average_body_count'], 10))
print(get_statistics(X_copy, ['case__average_match_singleton'], 10))
print(get_statistics(X_copy, ['case__average_match_value'], 10))
print(get_statistics(X_copy, ['case__average_match_sequence'], 10))
print(get_statistics(X_copy, ['case__average_match_mapping'], 10))
print(get_statistics(X_copy, ['case__average_match_class'], 10))
print(get_statistics(X_copy, ['case__average_match_star'], 10))
print(get_statistics(X_copy, ['case__average_match_or'], 10))
print(get_statistics(X_copy, ['case__average_match_as'], 10))

# 2 FEATURES
print("--- TWO FEATURES ---")
#print(get_statistics(X_copy, ['import__average_imported_modules', 'import__module_imports_pct'], 10))

# 3 FEATURES
print("--- THREE FEATURES ---")
#print(get_statistics(X_copy, ['program__has_sub_dirs_with_code', 'program__has_packages', 'program__has_code_root_package'], 10))

In [ ]:
def normalize_datatypes(X:pd.DataFrame, y:pd.Series) -> (pd.DataFrame, pd.Series, [str]):
    X = pd.get_dummies(X)
    X = X.astype('float32')
    y = y.apply(lambda value: 0 if value == "BEGINNER" else 1) # EXPERT will be 1 and BEGINNER will be 0.
    y = y.astype('float32')
    columns_names = X.columns.tolist()
    return X, y, columns_names

X, y, TABLE_FEATURES = normalize_datatypes(X, y)
# Print information about the loaded table
print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')

#### Muestra la matriz de correlación de pearson entre las variables de la tabla.

In [ ]:
sns.heatmap(X.corr(), annot=True)

## 3 - Detección de valores atípicos (outliers)
## Univariate
## Analisis detallado de variables
Para cada una de las 11 variable (11 numericas) se hara un analisis detallado

### Variable number_of_cases (1/11)
Esta variable es de tipo numérica y nos indica el número de clausulas case en la sentencia. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 2 - 51. Con una media de 5.17.

In [ ]:
sns.stripplot(X['case__number_of_cases'])

### IMPORTANTE
Los valores maximos estan muy alejados del siguiente.

In [ ]:
print_outliers_for_df_column(X, 'case__number_of_cases')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 10.0 (Con un MC de 79 usamos Tuckey Extremo)

In [ ]:
X[X['case__number_of_cases'] > 10].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### Variable guards (2/11)
Esta variable representa el número de guards de la sentencia. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__guards'])

### IMPORTANTE

Ningún Match tiene Guard. Esta variable toma siempre 0 como valor.

### Variable average_body_count (3/11)
Esta variable representa el número medio de sentencias en el cuerpo de los case. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 1.0 - 3.0. Con una media de 1.91.

In [ ]:
sns.stripplot(X['case__average_body_count'])

In [ ]:
print_outliers_for_df_column(X, 'case__average_body_count')

In [ ]:
X[X['case__average_body_count'] ==  1].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Nada que destacar

### Variable average_match_value (4/11)
Esta variable es de tipo numérica y nos indica el número medio de cases del tipo Value. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0.5 - 1. Con una media de 0.71.

In [ ]:
sns.stripplot(X['case__average_match_value'])

In [ ]:
print_outliers_for_df_column(X, 'case__average_match_value')

In [ ]:
X[X['case__average_match_value'] == 1.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Nada que destacar

### Variable average_match_singleton (5/11)
Esta variable representa el número medio de cases del tipo Singleton. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__average_match_singleton'])

### IMPORTANTE 

No hay ningún Match que incluya un MatchSingleton. Esta variable toma siempre el valor 0

### Variable average_match_sequence (6/11)
Esta variable representa el número medio de cases del tipo Sequence. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__average_match_sequence'])

### IMPORTANTE 

No hay ningún Match que incluya un MatchSequence. Esta variable toma siempre el valor 0

### Variable average_match_mapping (7/11)
Esta variable representa el número medio de cases del tipo Mapping. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__average_match_mapping'])

### IMPORTANTE 

No hay ningún Match que incluya un MatchMapping. Esta variable toma siempre el valor 0

### Variable average_match_star (8/11)
Esta variable representa el número medio de cases del tipo Star. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__average_match_star'])

### IMPORTANTE 

No hay ningún Match que incluya un MatchStar. Esta variable toma siempre el valor 0

### Variable average_match_or (9/11)
Esta variable representa el número medio de cases del tipo Or. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0 - 0. Con una media de 0.

In [ ]:
sns.stripplot(X['case__average_match_or'])

### IMPORTANTE 

No hay ningún Match que incluya un MatchOr. Esta variable toma siempre el valor 0

### Variable average_match_class (10/11)
Esta variable es de tipo numérica y nos indica el número medio de cases del tipo Match. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0.0 - 0.052. Con una media de 0.0018.

In [ ]:
sns.stripplot(X['case__average_match_class'])

In [ ]:
print_outliers_for_df_column(X, 'case__average_match_class')

Describimos los valores de las variables de la tabla, cuando el valor de la variable es mayor que 0 (Todos los umbrales son 0)

In [ ]:
X[X['case__average_match_class'] > 0.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
El único distinto de 0 es un outlier, hay que revisarlo.

### Variable average_match_as (11/11)
Esta variable es de tipo numérica y nos indica el número medio de cases del tipo As. Como vimos en la descripción de la tabla esta varibale adopta valores en el rango 0.0 - 0.5. Con una media de 0.28.

In [ ]:
sns.stripplot(X['case__average_match_as'])

In [ ]:
print_outliers_for_df_column(X, 'case__average_match_as')

In [ ]:
X[X['case__average_match_as'] > 0.0].describe(percentiles=[.25, .50, .75], include = ['object', 'float', 'bool', 'int'])

### IMPORTANTE
Nada que destacar

## Multivariate

TODO: isolation forest algorithm

In [ ]:
from sklearn.ensemble import  IsolationForest

CONTAMINATION_FACTOR = 0.0012

isof_model = IsolationForest(contamination=CONTAMINATION_FACTOR, random_state=0)
isof_prediction = isof_model.fit_predict(X.to_numpy())
mask = isof_prediction == -1
multivar_outliers = X[mask].index.tolist()
full_table.iloc[multivar_outliers]

### IMPORTANTE
Hay que analizar el outlier detectado.